In [4]:
import pandas as pd
import datetime as dt
from dateutil import parser
import boto3
import os

In [2]:
df = pd.read_csv('../../bat-2020-data/bat_results_output_2021-03-21_12-34-26.csv')

In [8]:
def build_id_int(x):
    d1 = parser.parse(x)
    d2 = (d1 - dt.datetime(1970, 1, 1)).total_seconds()
    return int(d2)

In [3]:
df.head()

,fid,title,result,link
0,0,1967 Porsche 911S Soft-Window Targa,"\n Sold for $300,001 on 3/19/21 ...",https://bringatrailer.com/listing/1967-porsche...
1,1,1986 Ford RS200 Evolution,"\n Sold for $502,500 on 3/19/21 ...",https://bringatrailer.com/listing/1986-ford-rs...
2,2,21-Years-Owned 1996 Ferrari F355 Spider 6-Speed,"\n Sold for $93,000 on 3/18/21",https://bringatrailer.com/listing/1996-ferrari...
3,3,7k-Mile 2017 Dodge Viper GTS-R Commemorative E...,"\n Sold for $235,000 on 3/18/21 ...",https://bringatrailer.com/listing/2017-dodge-v...
4,4,"3,500-Mile 1993 Acura NSX 5-Speed","\n Sold for $159,000 on 3/17/21 ...",https://bringatrailer.com/listing/1993-acura-n...


In [18]:
path = '../../bat-2020-data/'

In [10]:
filenames = os.listdir('../../bat-2020-data/')

In [26]:
import time
file_ts_str = filenames[41][19:].split('.')[0]
file_ts = dt.datetime.strptime(file_ts_str, '%Y-%m-%d_%H-%M-%S')
print(time.mktime(file_ts.timetuple()))

1618536863.0


In [29]:
for filename in filenames:
    if filename[4:11] == 'results':
        file_ts_str = filename[19:].split('.')[0]
        file_ts = int(time.mktime(dt.datetime.strptime(file_ts_str, '%Y-%m-%d_%H-%M-%S').timetuple()))
        print(file_ts)

1616474061
1616808858
1616992460
1616776459
1617057261
1616981660
1617154460
1616538860
1616603659
1616420060
1618288462
1617316458
1618061661
1618493663
1616711659
1617856465
1618536863
1617122059
1617024861
1616646859
1616949259
1617716060
1616873663
1617921260
1617068059
1617597259
1617748458
1618623259
1618202063
1617208464
1617942859
1616376860
1616895259
1618277662
1617251660
1617392059
1618612461
1618580063
1616506460
1618180466
1618148069
1618526066
1617284059
1618007660
1616614459
1616700860
1616430859
1617510861
1617338059
1618104860
1617737661
1617802461
1616636060
1616852060
1618547661
1618353262
1617456859
1618169665
1617370460
1616560458
1617230059
1618504462
1617165267
1616884459
1616441659
1617899660
1617910459
1618515266
1618688074
1618677260
1617932059
1617986060
1616906063
1618407263
1617726860
1617413659
1616733258
1618256062
1616387660
1617467664
1618320863
1616938460
1616549660
1618094059
1618191265
1617035659
1616452461
1618018460
1618418069
1617381259
1616625260

In [32]:
for filename in filenames:
    if filename[4:11] == 'results':
        file_ts_str = filename[19:].split('.')[0]
        file_ts = int(time.mktime(dt.datetime.strptime(file_ts_str, '%Y-%m-%d_%H-%M-%S').timetuple()))
        #ts = build_id_int(file_ts)
        df = pd.read_csv(path+filename)
        df.fid = [str(file_ts) + '_' + str(df.iloc[i].fid) for i in range(len(df))]
        df.result = [df.iloc[i].result.replace('\n', '').strip() for i in range(len(df))]
        df.to_csv(path+'temp-out/'+filename, index=False)

In [37]:
session = boto3.Session(profile_name='default')
s3 = boto3.resource('s3')
dev_s3_client = session.client('s3')

In [38]:
for filename in filenames:
    if filename[4:11] == 'results':
        upfilename = path+'temp-out/'+filename
        upfile = s3.meta.client.upload_file(Filename = upfilename,
                                            Bucket='bat-2020',
                                            Key = filename)
        os.remove(path+'temp-out/'+filename)